In [4]:
//IMPORTER LA BIBLIOTHEQUE SQL SPARK
import org.apache.spark.sql.types._

//IMPORTER LA BIBLIOTHEQUE GRAPHFRAME
import org.graphframes._

//IMPORTER LA BIBLIOTHEQUE GRAPHX
import org.apache.spark.graphx._


In [5]:
//DEFINITION DE LA STRUCTURE DU FICHIER D'ENTREE POUR LES NOEUDS
val aeroports_struct_noeuds = StructType(StructField("Code_comp", StringType, true) :: StructField("num_comp", StringType, true) :: StructField("code_source", StringType, true) :: StructField("id", StringType, true) :: StructField("code_dest", StringType, true) :: StructField("num_dest", StringType, true) :: StructField("filler", StringType, true) :: StructField("nb_etapes", ShortType, true) :: StructField("code", StringType, true) :: Nil)

//DEFINITION DE LA STRUCTURE DU FICHIER D'ENTREE POUR LES LIENS
val aeroports_struct_liens = StructType(StructField("Code_comp", StringType, true) :: StructField("num_comp", StringType, true) :: StructField("code_source", StringType, true) :: StructField("src", StringType, true) :: StructField("code_dest", StringType, true) :: StructField("dst", StringType, true) :: StructField("filler", StringType, true) :: StructField("nb_etapes", ShortType, true) :: StructField("code", StringType, true) :: Nil)

//LECTURE DU FICHIER D'ENTREE A L'AIDE DE LA STRUCTURE DES NOEUDS DEFINIE PLUS HAUT
val aeroports_noeuds = spark.read.format("csv").schema(aeroports_struct_noeuds).load("routes.dat")

//CREATION DE LA TABLE ASSOCIEE AUX NOEUDS
aeroports_noeuds.createOrReplaceTempView("aeroports_noeuds")

//SELECTION DES DONNEES SERVANT A LA CONSTRUCTION DU GRAPHE A PARTIR DE LA TABLE DES NOEUDS
val Noeuds = spark.sql("select distinct id, code_source from (select id, code_source from aeroports_noeuds where id not like '%N' and code_source not like '%N' union select num_dest, code_dest from aeroports_noeuds)")
Noeuds.createOrReplaceTempView("Noeuds")

//LECTURE DU FICHIER D'ENTREE A L'AIDE DE LA STRUCTURE DES LIENS DEFINIE PLUS HAUT
val aeroports_liens = spark.read.format("csv").schema(aeroports_struct_liens).load("routes.dat")

//CREATION DE LA TABLE ASSOCIEE AUX LIENS
aeroports_liens.createOrReplaceTempView("aeroports_liens")

//SELECTION DES DONNEES SERVANT A LA CONSTRUCTION DU GRAPHE A PARTIR DE LA TABLE DES LIENS
val Liens = spark.sql("select distinct src, code_source, dst, code_dest from aeroports_liens where src not like '%N' and dst not like '%N'")
Liens.createOrReplaceTempView("Liens")

//CONSTRUCTION DU GRAPHE
val Graphe_aeroports = GraphFrame(Noeuds, Liens)


aeroports_struct_noeuds = StructType(StructField(Code_comp,StringType,true), StructField(num_comp,StringType,true), StructField(code_source,StringType,true), StructField(id,StringType,true), StructField(code_dest,StringType,true), StructField(num_dest,StringType,true), StructField(filler,StringType,true), StructField(nb_etapes,ShortType,true), StructField(code,StringType,true))
aeroports_struct_liens = StructType(StructField(Code_comp,StringType,true), StructField(num_comp,StringType,true), StructField(code_source,StringType,true), StructField(src,StringType,true), StructField(code_dest,StringType,true), StructField(dst,StringType,true), StructField(filler,StringType,true), StructField(nb_etapes,ShortType,true...


StructType(StructField(Code_comp,StringType,true), StructField(num_comp,StringType,true), StructField(code_source,StringType,true), StructField(src,StringType,true), StructField(code_dest,StringType,true), StructField(dst,StringType,true), StructField(filler,StringType,true), StructField(nb_etapes,ShortType,true), StructField(code,StringType,true))

In [6]:
//POIDS DES DIFFERENTES LIGNES EN FONCTION DU NOMBRE DE COMPAGNIES EXPLOITANTES (INDEPENDAMMENT DU SENS DU PARCOURS)
val Poids_Liens = spark.sql("select noeud_1 as aeroport_1, noeud_2 as aeroport_2, count(*) as poids from (select substr(code,1,3) as noeud_1, substr(code,4,3) as noeud_2 from (select case when code_source < code_dest then concat(code_source,code_dest) else concat(code_dest,code_source) end as code from aeroports_liens where src not like '%N' and dst not like '%N' order by code)) group by aeroport_1, aeroport_2 order by poids desc, aeroport_1 asc")
Poids_Liens.createOrReplaceTempView("Poids_Liens")
val Poids = spark.sql("select poids from Poids_Liens")
Poids.describe().show

[Stage 16:=================================================>    (185 + 4) / 200]+-------+------------------+
|summary|             poids|
+-------+------------------+
|  count|             19080|
|   mean|3.5241090146750524|
| stddev| 2.566571683526507|
|    min|                 1|
|    max|                39|
+-------+------------------+



Poids_Liens = [aeroport_1: string, aeroport_2: string ... 1 more field]
Poids = [poids: bigint]


[poids: bigint]

In [23]:
val nb_noeuds = spark.sql("select cast(count(*) as decimal(7,2)) as nb_noeuds from Noeuds")
val nb_liens = spark.sql("select cast(count(*) as decimal(7,2)) as nb_liens from Poids_Liens")
val table_densite = nb_noeuds.crossJoin(nb_liens)
table_densite.createOrReplaceTempView("table_densite")
val densite = spark.sql("select cast((2*nb_liens)/(nb_noeuds*(nb_noeuds-1)) as decimal (5,4)) as densite from table_densite")



nb_noeuds = [nb_noeuds: decimal(7,2)]
nb_liens = [nb_liens: decimal(7,2)]
table_densite = [nb_noeuds: decimal(7,2), nb_liens: decimal(7,2)]
densite = [densite: decimal(5,4)]


[densite: decimal(5,4)]

In [8]:
//DISTRIBUTIONS DES DEGRES (PLUS PROCHES VOISINS)
val degres = Graphe_aeroports.degrees
degres.createOrReplaceTempView("degres")
spark.sql("select a.id, b.code_source as code_IATA, a.degree as degre_total from degres a inner join Noeuds b on b.id = a.id order by degree desc").show(5)
spark.sql("select degree from degres").describe().show


[Stage 28:===================================================>  (189 + 4) / 200]+----+---------+-----------+
|  id|code_IATA|degre_total|
+----+---------+-----------+
| 340|      FRA|        477|
|1382|      CDG|        470|
| 580|      AMS|        463|
|1701|      IST|        453|
|3682|      ATL|        433|
+----+---------+-----------+
only showing top 5 rows

[Stage 31:=================================================>    (183 + 4) / 200]+-------+------------------+
|summary|            degree|
+-------+------------------+
|  count|              3330|
|   mean|22.386786786786786|
| stddev| 48.81451894696512|
|    min|                 1|
|    max|               477|
+-------+------------------+



[id: string, degree: int]

degres = [id: string, degree: int]


lastException: Throwable = null


In [9]:
//CLUSTERING
//Détermination/distribution du nombre de triangles
val triangles = Graphe_aeroports.triangleCount.run
triangles.createOrReplaceTempView("triangles")
spark.sql("select * from triangles order by count desc").show
spark.sql("select sum(count) as total_triangles from triangles").show
spark.sql("select count from triangles").describe().show


[Stage 58:====================================================> (195 + 4) / 200]+-----+----+-----------+
|count|  id|code_source|
+-----+----+-----------+
| 4543| 580|        AMS|
| 4357| 340|        FRA|
| 4136|1382|        CDG|
| 3658| 346|        MUC|
| 3154| 507|        LHR|
| 3148|1701|        IST|
| 3142|1555|        FCO|
| 3094|1218|        BCN|
| 2755|1229|        MAD|
| 2665|3682|        ATL|
| 2652|1678|        ZRH|
| 2649| 302|        BRU|
| 2645| 599|        DUB|
| 2644|3830|        ORD|
| 2601|3364|        PEK|
| 2569|3797|        JFK|
| 2414|1613|        VIE|
| 2405|2188|        DXB|
| 2387|3494|        EWR|
| 2351| 345|        DUS|
+-----+----+-----------+
only showing top 20 rows

[Stage 81:====================================================> (195 + 4) / 200]+---------------+
|total_triangles|
+---------------+
|         302511|
+---------------+

[Stage 105:==================================================>  (192 + 4) / 200]+-------+------------------+
|summary|     

triangles = [count: bigint, id: string ... 1 more field]


[count: bigint, id: string ... 1 more field]

In [10]:
//Clustering détaillé
val clustering = spark.sql("select  t.id, t.code_source, t.count as nb_triangles, d.degree as nb_degres, cast((2*t.count)/(d.degree*(d.degree - 1)) as decimal(7,5) ) as clustering from triangles t inner join degres d on d.id = t.id")
clustering.createOrReplaceTempView("clustering")
spark.sql("select * from clustering where nb_degres > 318 and nb_triangles > 3029 order by clustering desc").show
spark.sql("select clustering from clustering").describe().show


[Stage 131:====================================================>(198 + 2) / 200]+----+-----------+------------+---------+----------+
|  id|code_source|nb_triangles|nb_degres|clustering|
+----+-----------+------------+---------+----------+
|1218|        BCN|        3094|      326|   0.05840|
| 507|        LHR|        3154|      340|   0.05473|
| 346|        MUC|        3658|      380|   0.05080|
| 580|        AMS|        4543|      463|   0.04248|
| 340|        FRA|        4357|      477|   0.03838|
|1382|        CDG|        4136|      470|   0.03753|
|1701|        IST|        3148|      453|   0.03075|
+----+-----------+------------+---------+----------+

[Stage 156:===================================================> (194 + 4) / 200]+-------+------------------+
|summary|        clustering|
+-------+------------------+
|  count|              3308|
|   mean|       0.136940641|
| stddev|0.1754712410326138|
|    min|           0.00000|
|    max|           1.00000|
+-------+---------------

clustering = [id: string, code_source: string ... 3 more fields]


[id: string, code_source: string ... 3 more fields]

In [25]:
spark.sql("select id,clustering from clustering order by clustering desc").coalesce(1).write.format("com.databricks.spark.csv").option("header","true").save("clustering.csv")

[Stage 3438:==================================================> (196 + 4) / 200]

lastException: Throwable = null


In [11]:
//Clustering global
val somme_clustering = spark.sql("select cast(sum(clustering) as decimal(7,2)) as somme_clustering from clustering")
val nb_noeuds = spark.sql("select cast(count(*) as decimal(7,2)) as nb_noeuds from Noeuds")
val table_clustering_global = somme_clustering.crossJoin(nb_noeuds)
table_clustering_global.createOrReplaceTempView("table_clustering_global")
spark.sql("select cast(somme_clustering/nb_noeuds as decimal(5,4)) as clustering_global from table_clustering_global").show


[Stage 185:===================================================> (195 + 5) / 200]+-----------------+
|clustering_global|
+-----------------+
|           0.1323|
+-----------------+



somme_clustering = [somme_clustering: decimal(7,2)]
nb_noeuds = [nb_noeuds: decimal(7,2)]
table_clustering_global = [somme_clustering: decimal(7,2), nb_noeuds: decimal(7,2)]


[somme_clustering: decimal(7,2), nb_noeuds: decimal(7,2)]

In [12]:
//COMPOSANTES CONNEXES
val GrapheX_aeroports = Graphe_aeroports.toGraphX
val composants_connexes = GrapheX_aeroports.connectedComponents
def sortedConnectedComponents(connectedComponents: Graph[VertexId, _]): Seq[(VertexId, Long)] = {
  val componentCounts = connectedComponents.vertices.map(_._2).countByValue
  componentCounts.toSeq.sortBy(_._2).reverse
}
val componentCounts = sortedConnectedComponents( composants_connexes)
componentCounts.size
componentCounts.take(11).foreach(println)


(0,3304)                                                                        
(300647710722,10)
(266287972363,4)
(9,4)
(352187318278,4)
(523986010129,2)
(438086664202,2)
(326417514507,1)
(206158430216,1)
(687194767370,1)
(601295421446,1)


GrapheX_aeroports = org.apache.spark.graphx.impl.GraphImpl@439972eb
composants_connexes = org.apache.spark.graphx.impl.GraphImpl@77af378c
componentCounts = ArrayBuffer((0,3304), (300647710722,10), (266287972363,4), (9,4), (352187318278,4), (523986010129,2), (438086664202,2), (326417514507,1), (206158430216,1), (687194767370,1), (601295421446,1))


sortedConnectedComponents: (connectedComponents: org.apache.spark.graphx.Graph[org.apache.spark.graphx.VertexId, _])Seq[(org.apache.spark.graphx.VertexId, Long)]


ArrayBuffer((0,3304), (300647710722,10), (266287972363,4), (9,4), (352187318278,4), (523986010129,2), (438086664202,2), (326417514507,1), (206158430216,1), (687194767370,1), (601295421446,1))

In [13]:
val connexion_principale = componentCounts.take(1).toList.toDF
connexion_principale.createOrReplaceTempView("connexion_principale")
val nb_noeuds_connectes = spark.sql("select _2 as nb_noeuds_connectes from connexion_principale")
nb_noeuds_connectes.show
val pourcentage_noeuds_connexion_principale = nb_noeuds_connectes.crossJoin(nb_noeuds)
pourcentage_noeuds_connexion_principale.createOrReplaceTempView("pourcentage_noeuds_connexion_principale")
spark.sql("select cast((nb_noeuds_connectes/nb_noeuds)*100 as decimal(4,2)) as pourcentage_noeuds_connectes from pourcentage_noeuds_connexion_principale").show


+-------------------+
|nb_noeuds_connectes|
+-------------------+
|               3304|
+-------------------+

[Stage 426:============================================>        (169 + 4) / 200]+----------------------------+
|pourcentage_noeuds_connectes|
+----------------------------+
|                       96.47|
+----------------------------+



connexion_principale = [_1: bigint, _2: bigint]
nb_noeuds_connectes = [nb_noeuds_connectes: bigint]
pourcentage_noeuds_connexion_principale = [nb_noeuds_connectes: bigint, nb_noeuds: decimal(7,2)]


[nb_noeuds_connectes: bigint, nb_noeuds: decimal(7,2)]

In [14]:
//CENTRALITES - RECHERCHE DE PLUS COURTS TRAJETS AVEC AEROPORTS INTERMEDIAIRES
val trajets = Graphe_aeroports.find("(a)-[]->(b); (b)-[]->(c)")
trajets.show()
trajets.createOrReplaceTempView("trajets")
//Récupération des aéroports intermédiaires sur le plus court trajet
val intermediaires = spark.sql("select b.code_source from trajets where a<>b and b<>c")
intermediaires.createOrReplaceTempView("intermediaires")
val centralites = spark.sql("select code_source as code_IATA, count(*) as centralite from intermediaires group by code_source order by centralite desc")
centralites.createOrReplaceTempView("centralites")
centralites.show(5)
spark.sql("select centralite from centralites").describe().show


[Stage 436:==================================================>  (192 + 4) / 200]+----------+----------+----------+
|         a|         b|         c|
+----------+----------+----------+
|[2397,MNL]|[2426,MPH]|[2397,MNL]|
|[2397,MNL]|[2426,MPH]|[4206,CEB]|
| [346,MUC]|[1558,EBA]| [382,FDH]|
| [346,MUC]|[1558,EBA]|[1676,BRN]|
| [346,MUC]|[1558,EBA]|[1679,ACH]|
| [346,MUC]|[1558,EBA]|[1678,ZRH]|
| [346,MUC]|[1558,EBA]| [346,MUC]|
|[3682,ATL]|[5756,MSL]|[3682,ATL]|
|[4108,TNA]|[3382,KMG]|[6347,SJW]|
|[4108,TNA]|[3382,KMG]|[6361,YIH]|
|[4108,TNA]|[3382,KMG]|[6944,JNG]|
|[4108,TNA]|[3382,KMG]|[3179,HKT]|
|[4108,TNA]|[3382,KMG]|[3043,CCU]|
|[4108,TNA]|[3382,KMG]|[3386,HGH]|
|[4108,TNA]|[3382,KMG]|[3304,KUL]|
|[4108,TNA]|[3382,KMG]|[6399,LZO]|
|[4108,TNA]|[3382,KMG]|[3370,CAN]|
|[4108,TNA]|[3382,KMG]|[4149,XUZ]|
|[4108,TNA]|[3382,KMG]|[3388,NKG]|
|[4108,TNA]|[3382,KMG]|[3376,WUH]|
+----------+----------+----------+
only showing top 20 rows

+---------+----------+                                

trajets = [a: struct<id: string, code_source: string>, b: struct<id: string, code_source: string> ... 1 more field]
intermediaires = [code_source: string]
centralites = [code_IATA: string, centralite: bigint]


[code_IATA: string, centralite: bigint]

In [31]:
centralites.coalesce(1).write.format("com.databricks.spark.csv").option("header","true").save("centralite.csv")


[Stage 3489:===================================================>(199 + 1) / 200]

lastException: Throwable = null


In [15]:
//LPA (LABEL PROPAGATION) - DETERMINATION DE L'EXISTENCE DE COMMUNAUTES
val LPA = Graphe_aeroports.labelPropagation.maxIter(5).run()
LPA.createOrReplaceTempView("LPA")
LPA.select("id", "label").show()


[Stage 605:=================================================>   (188 + 4) / 200]+----+-------------+
|  id|        label|
+----+-------------+
| 468| 549755813898|
|1972| 695784701963|
|5575| 549755813898|
|1909| 549755813898|
|8314|1030792151047|
|2688|1374389534720|
|8259|1494648619016|
|2256| 420906795015|
|1925|1030792151047|
|6157| 549755813898|
|6135|1030792151047|
|3514|1030792151047|
|5492|           16|
|2001|1056561954834|
|2517| 395136991234|
|7150|1125281431570|
|3782|1030792151047|
|6265|1185410973711|
|6992|1030792151047|
|2917|  51539607564|
+----+-------------+
only showing top 20 rows



LPA = [id: string, code_source: string ... 1 more field]


[id: string, code_source: string ... 1 more field]

In [16]:
//Les 20 groupes les plus importants
val communautes = spark.sql("select label, count(*) as nbre from LPA group by label order by nbre desc")
communautes.createOrReplaceTempView("communautes")
val LPA_stat = spark.sql("select nbre from communautes")
LPA_stat.describe().show
communautes.show(20)


+-------+------------------+                                                    
|summary|              nbre|
+-------+------------------+
|  count|               339|
|   mean|10.103244837758112|
| stddev| 56.40873097551725|
|    min|                 1|
|    max|               736|
+-------+------------------+

[Stage 717:===================================================> (195 + 5) / 200]+-------------+----+
|        label|nbre|
+-------------+----+
| 549755813898| 736|
|1030792151047| 675|
| 790273982469| 211|
| 206158430212| 123|
|  51539607564| 110|
| 326417514507|  92|
| 171798691842|  85|
|1374389534733|  81|
| 420906795015|  77|
| 317827579911|  28|
|1151051235329|  28|
|1400159338500|  27|
| 498216206341|  25|
| 498216206337|  23|
| 584115552258|  23|
| 420906795025|  23|
| 257698037777|  22|
|  25769803782|  22|
| 558345748488|  21|
| 395136991234|  21|
+-------------+----+
only showing top 20 rows



communautes = [label: bigint, nbre: bigint]
LPA_stat = [nbre: bigint]


[nbre: bigint]

In [17]:
spark.sql("select distinct l.id, a.code_source, c.nb_triangles, c.nb_degres from LPA l inner join aeroports_noeuds a on a.id = l.id inner join clustering c on c.id = l.id where label = 51539607564 order by c.nb_degres desc, c.nb_triangles desc ").show
//549755813898 (736)  => Europe
//1030792151047 (675) => Amérique du Nord
//790273982469 (211)  => Chine
//206158430212 (123)  => Moyen Orient - Inde - Pakistan
//51539607564 (110)   => Asie du Nord (Russie et anciennes républiques de l'Union soviétique) 
//420906795015 (77)   => Extrême Orient (Viêt-Nam)
//Le nombre total de labels est égal au nombre de noeuds du réseau
spark.sql("select sum(nbre) from(select label, count(*) as nbre from LPA group by label order by nbre desc)").show


[Stage 770:====================================================>(198 + 2) / 200]+----+-----------+------------+---------+
|  id|code_source|nb_triangles|nb_degres|
+----+-----------+------------+---------+
|2975|        SVX|         416|      130|
|2983|        TAS|         442|      113|
|4078|        OVB|         355|      103|
|2908|        ALA|         275|       90|
|2910|        TSE|         169|       68|
|2979|        DYU|         142|       61|
|4374|        KJA|         153|       57|
|2937|        IKT|          89|       52|
|2912|        FRU|         118|       44|
|2990|        KZN|          90|       42|
|2927|        KHV|          78|       42|
|2923|        YKS|          60|       40|
|2992|        UFA|          74|       38|
|6147|        LBD|          63|       38|
|4111|        TJM|          53|       37|
|2965|        AER|          79|       35|
|4367|        SCO|          46|       34|
|2934|        VVO|          49|       30|
|4274|        GOJ|          48|       

In [18]:
//TABLES POUR CALCUL MODULARITE
val table_Q1 = spark.sql("select a.src, b.code_source as code_src, d.degree as deg_src, f.label as lab_src, a.dst, c.code_source as code_dst, e.degree as deg_dst, g.label as lab_dst from Liens a inner join Noeuds b on b.id = a.src inner join Noeuds c on c.id = a.dst inner join degres d on d.id = a.src inner join degres e on e.id = a.dst inner join LPA f on f.id = a.src inner join LPA g on g.id = a.dst")
table_Q1.createOrReplaceTempView("table_Q1")
val nb_tot_liens = spark.sql("select cast(count(*) as decimal(6,1)) as nb_tot_liens from Poids_Liens")
val table_Q2 = table_Q1.crossJoin(nb_tot_liens)
table_Q2.createOrReplaceTempView("table_Q2")
val table_Q3bis = spark.sql("select lab_src, lab_dst, deg_src, deg_dst, cast((deg_src*deg_dst)/(2*nb_tot_liens) as decimal (5,2)) as produit from table_Q2")
table_Q3bis.createOrReplaceTempView("table_Q3bis")
val somme_produit = spark.sql("select cast(sum(produit) as decimal(7,2)) as somme_produit from table_Q3bis")
somme_produit.createOrReplaceTempView("somme_produit")
val table_Q4 = somme_produit.crossJoin(nb_tot_liens)
table_Q4.createOrReplaceTempView("table_Q4")
spark.sql("select cast(1-((somme_produit)/(2*nb_tot_liens)) as decimal (5,4)) as modularite from table_Q4").show


[Stage 873:===================================================> (194 + 4) / 200]+----------+
|modularite|
+----------+
|    0.5806|
+----------+



table_Q1 = [src: string, code_src: string ... 6 more fields]
nb_tot_liens = [nb_tot_liens: decimal(6,1)]
table_Q2 = [src: string, code_src: string ... 7 more fields]
table_Q3bis = [lab_src: bigint, lab_dst: bigint ... 3 more fields]
somme_produit = [somme_produit: decimal(7,2)]
table_Q4 = [somme_produit: decimal(7,2), nb_tot_liens: decimal(6,1)]


[somme_produit: decimal(7,2), nb_tot_liens: decimal(6,1)]

In [19]:
//Plus court chemin entre deux aéroports
val a : String = "340"
val b : String = "5557" 
val plus_court_chemin = Graphe_aeroports.shortestPaths.landmarks(Seq(a, b)).run()
plus_court_chemin.select("id", "distances").show()


[Stage 1130:==================================================> (196 + 4) / 200]+----+--------------------+
|  id|           distances|
+----+--------------------+
| 468|Map(340 -> 2, 555...|
|1972|Map(340 -> 3, 555...|
|5575|Map(340 -> 3, 555...|
|1909|Map(340 -> 1, 555...|
|8314|Map(340 -> 2, 555...|
|2688|Map(340 -> 2, 555...|
|8259|               Map()|
|2256|Map(340 -> 2, 555...|
|1925|Map(340 -> 3, 555...|
|6157|Map(340 -> 2, 555...|
|6135|Map(340 -> 3, 555...|
|3514|Map(340 -> 2, 555...|
|5492|Map(340 -> 4, 555...|
|2001|               Map()|
|2517|Map(340 -> 3, 555...|
|7150|Map(340 -> 4, 555...|
|3782|Map(340 -> 2, 555...|
|6265|Map(340 -> 4, 555...|
|6992|Map(340 -> 2, 555...|
|2917|Map(340 -> 2, 555...|
+----+--------------------+
only showing top 20 rows



a = 340
b = 5557
plus_court_chemin = [id: string, code_source: string ... 1 more field]


[id: string, code_source: string ... 1 more field]

In [20]:
//Strongly connected
val SC = Graphe_aeroports.stronglyConnectedComponents.maxIter(10).run()
SC.select("id", "component").orderBy("component").show()


+----+---------+                                                                
|  id|component|
+----+---------+
|6733|        0|
|1579|        0|
|5575|        0|
| 468|        0|
|5475|        0|
|2688|        0|
|2256|        0|
|1925|        0|
|1972|        0|
|8314|        0|
|3514|        0|
|5492|        0|
|2517|        0|
|7150|        0|
|3782|        0|
|6265|        0|
|6992|        0|
|6157|        0|
|1909|        0|
|6135|        0|
+----+---------+
only showing top 20 rows



SC = [id: string, code_source: string ... 1 more field]


[id: string, code_source: string ... 1 more field]